## Data Embedding

##### 사용한 pennylane template: https://docs.pennylane.ai/en/stable/introduction/templates.html

In [1]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

In [2]:
qubit_num = 3

## 1. Basis Embedding

### 1.1. Basis Embedding with Pennylane Templates

In [3]:
dev = qml.device("default.qubit", wires=qubit_num)

@qml.qnode(dev)
def basis_embedding_circuit(feature_vector):
    qml.BasisEmbedding(features=feature_vector, wires=range(qubit_num))
    return qml.state()

vector = [1, 0, 1]

In [28]:
print(qml.draw(basis_embedding_circuit, expansion_strategy="device")(vector))

0: ──X─┤  State
2: ──X─┤  State


In [5]:
to_vec_bin_str = [np.binary_repr(i, width=qubit_num) for i in range(2 ** qubit_num)]
vectors = [[int(c) for c in bin_str] for bin_str in to_vec_bin_str]

for vec in vectors:
    print(basis_embedding_circuit(vec))
    print()


[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]



### 1.2. DIY Basis Embedding

In [6]:
dev = qml.device("default.qubit", wires=qubit_num)

@qml.qnode(dev)
def Custom_Basis_Embedding(data: int, width: int):
    bin_str = np.binary_repr(data, width=width)
    for i in range(width):
        if (bin_str[i]=='1'):
            qml.PauliX(wires=i)
    return qml.state()

for i in range(2 ** qubit_num):
    print(f"Output state = {Custom_Basis_Embedding(i, width=qubit_num)}\n")

Output state = [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

Output state = [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

Output state = [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

Output state = [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]

Output state = [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]

Output state = [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]

Output state = [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]

Output state = [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]



## 2. Amplitude Embedding

### 2.1. Amplitude Embedding with Pennylane Templates

In [7]:
@qml.qnode(dev)
def Amplitude_Embedding_Circuit(f=None):
    qml.AmplitudeEmbedding(features=f, wires=range(qubit_num), normalize=True)
    return qml.expval(qml.PauliZ(0))

Amplitude_Embedding_Circuit(f=[1 for i in range(2 ** qubit_num)])

tensor(0., requires_grad=True)

In [8]:
dev.state

tensor([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
        0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j], requires_grad=True)

### 2.2. DIY Amplitude Embedding

In [21]:
@qml.qnode(dev)
def Custom_Amplitude_Embedding_Circuit(feature: list, normalize=True, pad_num = 0):
    # if padding is needed
    if int(np.log2(len(feature))) != np.log2(len(feature)):
        feature.extend([pad_num for i in range(2 ** int(np.log2(len(feature))) + 1) - len(feature)])
    
    # Normalize
    if normalize == True:
        feature = np.array(feature)
        normalized_feature = feature / np.sqrt(np.sum(feature ** 2))

    qml.QubitStateVector(normalized_feature, wires=range(qubit_num))
    return qml.expval(qml.PauliZ(0))

In [22]:
Custom_Amplitude_Embedding_Circuit(feature=[1 for i in range(2 ** qubit_num)], normalize=True, pad_num=0)

tensor(0., requires_grad=True)

In [23]:
dev.state

tensor([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
        0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j], requires_grad=True)

## 3.AngleEmbedding

### 3.1. Angle Embedding with Pennylane Templates

In [10]:
@qml.qnode(dev)
def Angle_Embedding_Circuit(feature_vector):
    qml.AngleEmbedding(features=feature_vector, wires=range(qubit_num), rotation="X")
    qml.Hadamard(wires=0)
    return qml.probs(wires=range(qubit_num))

feature = [1, 2, 3]

In [11]:
print(qml.draw(Angle_Embedding_Circuit, expansion_strategy="device")(feature))

0: ──RX(1.00)──H─┤ ╭Probs
1: ──RX(2.00)────┤ ├Probs
2: ──RX(3.00)────┤ ╰Probs


### 3.2. DIY Angle Embedding

In [12]:
@qml.qnode(dev)
def Custom_Angle_Embedding_Circuit(feature_vector, rotation="X", *rotargs):
    for i in range(len(feature_vector)):
        if (rotation == "X"):
            qml.RX(feature_vector[i], wires=i)
        elif (rotation == "Y"):
            qml.RY(feature_vector[i], wires=i)
        else:
            qml.Rot(rotargs)
    
    return qml.probs(wires=range(qubit_num))

feature=[3, -1, 2]

In [27]:
print(qml.draw(Custom_Angle_Embedding_Circuit, expansion_strategy="device")(feature))

0: ──RX(3.00)──┤ ╭Probs
1: ──RX(-1.00)─┤ ├Probs
2: ──RX(2.00)──┤ ╰Probs
